In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from langchain_text_splitters import RecursiveJsonSplitter

In [2]:
MODEL = "gpt-4o-mini"
json_path = "C:/Users/aalperen.arda/Documents/GitHub/LLM-Biography-Analysis/main/structured_bios_new_english_fixed.json"

In [ ]:
documents = []

with open(json_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)

for person in json_data:
    content = json.dumps(person, ensure_ascii=False)
    doc = Document(page_content=content, metadata={"ad": person["ad"]})
    documents.append(doc)

243 adet Document oluşturuldu.
{"ad": "Ahmet Doğan", "dogum_yeri": "Kayseri", "dogum_tarihi": "15.03.1988", "yas": 37, "ilkokul": "Hacı Bektaş-ı Veli İlkokulu", "ilkokul_yillari": "1994-2002", "lise": "Ankara Fen Lisesi", "lise_yillari": "2002-2006", "universite": "Orta Doğu Teknik Üniversitesi (ODTÜ)", "universite_yillari": "2006-2010", "bolum": "Bilgisayar Mühendisliği", "yuksek_lisans": "Koç Üniversitesi", "yuksek_lisans_yillari": "2010-2012", "doktora": null, "doktora_yillari": null, "calistigi_kurumlar": ["ASELSAN", "SAP Türkiye", "Artemis Yazılım"], "calisma_yillari": ["2010-2014", "2014-2017", "2017-devam ediyor"], "kurdugu_girisim_ve_dernekler": ["Artemis Yazılım"], "girisim_kurulus_yillari": ["2017"], "yasadigi_sehir": "İstanbul", "hobiler": ["Tarihi roman okumak", "amatör drone fotoğrafçılığı", "bağlama çalmak", "açık kaynak yazılım projelerine katkıda bulunmak"], "es": "Elif Doğan", "cocuklar": ["Deniz Doğan", "Ada Doğan"], "akademik_yayinlar": [], "yayin_yillari": [], "dosy

In [ ]:
'''text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"{len(chunks)} adet chunk oluşturuldu.")'''

# chunk oluşturmaya gerek yok, zaten tüm doc elementler yeterince küçük

'text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)\nchunks = text_splitter.split_documents(documents)\n\nprint(f"{len(chunks)} adet chunk oluşturuldu.")'

In [ ]:
embeddings = OpenAIEmbeddings()
persist_dir = "C:/Users/aalperen.arda/Documents/GitHub/LLM-Biography-Analysis/main/chroma_vector_store"

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_dir
)

In [19]:
data = vectorstore._collection.get(include=["embeddings", "documents", "metadatas"])

embeddings = data["embeddings"]
texts = data["documents"]
names = [m.get("ad", f"doc_{i}") for i, m in enumerate(data["metadatas"])]

In [24]:
X = np.array(embeddings)

cos_sim_matrix = cosine_similarity(X)

k_word = 30
pairs = set()

for i in range(len(cos_sim_matrix)):
    top_indices = np.argsort(cos_sim_matrix[i])[::-1][1:k_word + 1]
    
    for j in top_indices:
        pair = tuple(sorted((i, j)))
        pairs.add(pair) 

print(f"{len(pairs)} benzersiz çift eşleştirmesi üretildi.")

5404 benzersiz çift eşleştirmesi üretildi.


In [ ]:
client = OpenAI()
relationship_results = []

for i, j in tqdm(pairs, desc="LLM ile ilişki çıkarılıyor"):
    prompt = f"""
Aşağıdaki iki kişinin biyografisine dayanarak, aralarında hangi ilişkiler olabilir? Yalnızca aşağıdaki seçeneklerden uygun olanları belirt:

- AYNI_MEMLEKETTEN
- OKUL_ARKADAŞI
- MESLEKTAŞ
- YOK

Sadece ilişki adlarını virgülle yaz.

Kişi 1:
{texts[i]}

Kişi 2:
{texts[j]}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        result = response.choices[0].message.content.strip().upper()
        relations = [r.strip() for r in result.split(",") if r and r != "YOK"]

        for rel in relations:
            relationship_results.append({
                "source": names[i],
                "target": names[j],
                "relation": rel
            })

    except Exception as e:
        print(f"HATA ({names[i]} ↔ {names[j]}):", e)

In [ ]:
output_path = os.path.join(persist_dir, "llm_relationships.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(relationship_results, f, ensure_ascii=False, indent=2)

print(f"{len(relationship_results)} adet ilişki çıkarıldı ve kaydedildi → {output_path}")